In [1]:
from MySQLdb import connect
import pandas as pd

/var/folders/fb/xl0hnskx527bfqkv38l3sn7h0000gn/T/ipykernel_34795/3687392072.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Connect to your MySQLdb
#conn = connect(host='localhost', user='root')
conn = connect(host='127.0.0.1', user='root', password = 'root', port = 8890)

In [3]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [4]:
# Return data from the Customers table:
cur.execute('''SHOW DATABASES;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,Database
0,information_schema
1,mysql
2,performance_schema
3,sqlw3s
4,sys
5,testDB


The SQL SELECT INTO Statement
- The SELECT INTO statement copies data from one table into a new table.

>SELECT INTO Syntax
>
- Copy all columns into a new table:
>
>SELECT *
>
>INTO newtable [IN externaldb]
>
>FROM oldtable
>
>WHERE condition;


- Copy only some columns into a new table:
>
>SELECT column1, column2, column3, ...
>
>INTO newtable [IN externaldb]
>
>FROM oldtable
>
>WHERE condition;


The new table will be created with the column-names and types as defined in the old table. You can create new column names using the AS clause.

- SQL SELECT INTO Examples

In [5]:
# The following SQL statement creates a backup copy of Customers:
cur.execute('''SELECT * INTO CustomersBackup2017 FROM testDB.Customers;''')

OperationalError: (1327, 'Undeclared variable: CustomersBackup2017')

In [7]:
# The following SQL statement uses the IN clause to copy the table into a new table in another database:
cur.execute('''SELECT * INTO CustomersBackup2017 IN 'Backup.mdb' FROM testDB.Customers;''')

OperationalError: (1327, 'Undeclared variable: CustomersBackup2017')

In [8]:
# The following SQL statement copies only a few columns into a new table:
cur.execute('''SELECT CustomerName, ContactName INTO CustomersBackup2017 FROM testDB.Customers;''')

OperationalError: (1327, 'Undeclared variable: CustomersBackup2017')

In [9]:
# The following SQL statement copies only the German customers into a new table:
cur.execute('''SELECT * INTO CustomersGermany
FROM testDB.Customers
WHERE Country = 'Germany';''')

OperationalError: (1327, 'Undeclared variable: CustomersGermany')

In [15]:
# The following SQL statement copies data from more than one table into a new table:
cur.execute('''SELECT Customers.CustomerName, Orders.OrderID
INTO CustomersOrderBackup2017
FROM testDB.Customers
LEFT JOIN testDB.Orders ON Customers.CustomerID = Orders.CustomerID;''')

OperationalError: (1327, 'Undeclared variable: CustomersOrderBackup2017')

In [17]:
# Return data from the Customers table:
cur.execute('''SELECT Customers.CustomerName, Orders.OrderID
FROM testDB.Customers
LEFT JOIN testDB.Orders ON Customers.CustomerID = Orders.CustomerID;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,CustomerName,OrderID
0,Wilman Kala,10248.0
1,Tradição Hipermercados,10249.0
2,Hanari Carnes,10250.0
3,Victuailles en stock,10251.0
4,Suprêmes délices,10252.0
...,...,...
208,Paris spécialités,NaN
209,Rancho grande,NaN
210,Spécialités du monde,NaN
211,The Cracker Box,NaN


Tip: SELECT INTO can also be used to create a new, empty table using the schema of another.
- Just add a WHERE clause that causes the query to return no data:

>SELECT * INTO newtable
>
>FROM oldtable
>
>WHERE 1 = 0;

In [18]:
# V MySQL se musí použít CREATE TABLE table_name AS SELECT:
cur.execute('''CREATE TABLE testDB.CustomersOrderBackup2017
AS SELECT Customers.CustomerName, Orders.OrderID
FROM testDB.Customers
LEFT JOIN testDB.Orders ON Customers.CustomerID = Orders.CustomerID;''')

213

In [19]:
# Return data from the Customers table:
cur.execute('''SELECT * FROM testDB.CustomersOrderBackup2017;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,CustomerName,OrderID
0,Wilman Kala,10248.0
1,Tradição Hipermercados,10249.0
2,Hanari Carnes,10250.0
3,Victuailles en stock,10251.0
4,Suprêmes délices,10252.0
...,...,...
208,Paris spécialités,NaN
209,Rancho grande,NaN
210,Spécialités du monde,NaN
211,The Cracker Box,NaN
